In [1]:
import pdfplumber
import csv

In [2]:
pdf = pdfplumber.open("mttq_vietcombank_12_9.pdf")

In [7]:
import re
import csv

with open('mttq.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['transaction_code', 'amount', 'transaction_detail']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    start = False
    current_transaction = None

    for page in pdf.pages:  # Process each page in the PDF
        text_chunks = page.extract_text().split('\n')  # Extract text and split by line

        for line in text_chunks:
            if not start:
                if 'Số tài khoản' in line:  # Start processing after 'Số tài khoản'
                    start = True
                continue  # Skip lines until 'Số tài khoản' is found

            # Check if line matches the transaction code pattern
            transaction_code_match = re.match(r'^\"?([0-9A-Za-z\.]+)\.(.+)', line)
            if transaction_code_match:
                if current_transaction:
                    # Write the current transaction to the CSV file
                    current_transaction['transaction_detail'] = ' '.join(current_transaction['transaction_detail_lines']).strip()
                    writer.writerow({
                        'transaction_code': current_transaction.get('transaction_code', ''),
                        'amount': current_transaction.get('amount', ''),
                        'transaction_detail': current_transaction.get('transaction_detail', '')
                    })

                # Start a new transaction
                current_transaction = {}
                transaction_code = transaction_code_match.group(1).replace('.', '')
                current_transaction['transaction_code'] = transaction_code
                current_transaction['transaction_detail_lines'] = []
                transaction_detail_line = transaction_code_match.group(2).strip()
                current_transaction['transaction_detail_lines'].append(transaction_detail_line)

            else:
                # Check if line matches the STT, date, and amount pattern
                stt_date_amount_match = re.match(r'^(\d+)\s+(\d{2}/\d{2}/\d{4})\s+([\d\.]+)(.*)', line)
                if stt_date_amount_match:
                    if current_transaction:
                        # Store STT, date, and amount for the current transaction
                        current_transaction['STT'] = stt_date_amount_match.group(1)
                        current_transaction['date'] = stt_date_amount_match.group(2)
                        current_transaction['amount'] = stt_date_amount_match.group(3).replace('.', '')  # Remove periods from amount
                        remaining_text = stt_date_amount_match.group(4).strip()
                        if remaining_text:
                            current_transaction['transaction_detail_lines'].append(remaining_text)

                else:
                    # Append any additional details to the current transaction
                    if current_transaction:
                        current_transaction['transaction_detail_lines'].append(line.strip())

        # After processing all lines on the page, finalize the current transaction
        if current_transaction:
            current_transaction['transaction_detail'] = ' '.join(current_transaction['transaction_detail_lines']).strip()
            writer.writerow({
                'transaction_code': current_transaction.get('transaction_code', ''),
                'amount': current_transaction.get('amount', ''),
                'transaction_detail': current_transaction.get('transaction_detail', '')
            })
            current_transaction = None  # Reset current transaction

print("Finished processing.")


Finished processing.
